# Paragraph wise summarizer

In [1]:
# install libraries
! pip3 install llama-index
! pip3 install --upgrade llama_index
! pip3 install pypdf
! pip3 install --user -U nltk
! pip3 install --user -U numpy
! pip3 install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# import API keys
import os
import openai
from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")


In [2]:
# collect input text
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()

print(documents)

# nodes will contain two Node objects, one for each paragraph

/Users/presentations(work)/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(id_='59b57fa8-a94e-48bd-a0c8-c5466ca716bf', embedding=None, metadata={'page_label': '1', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='6cb3f6149429b2a9c15988e08db61900f23d65c38b0e91ab55a6ebabb4c830e3', text='    \u2029from Essays: First Series (1841)Type to enter textSelf-Reliance\n"Ne te quaesiveris extra." "Man is his own star; and the soul that can Render an honest and a perfect man, Commands all light, all inﬂuence, all fate; Nothing to him falls early or too late. Our acts our angels are, or good or ill, Our fatal shadows that walk by us still."       Epilogue to Beaumont and Fletcher\'s Honest Man\'s Fortune Cast the bantling on the rocks, Suckle him with the she-wolf\'s teat; Wintered with the hawk and fox, Power and speed be hands and feet.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_

In [3]:
# split by paragraph text splitter
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
text_splitter = SentenceSplitter(
    separator="\n\n", 
    paragraph_separator="\n\n\n"
)

In [4]:
# Make parsing more verbose - see whats going on under the hood
import logging
# Enable logging
logging.basicConfig(level=logging.INFO)


In [5]:
# parse split text into nodes
from llama_index.node_parser import SimpleNodeParser
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)



In [14]:
nodes

[TextNode(id_='98755f0f-004a-4dd4-b89e-7f6e01387226', embedding=None, metadata={'page_label': '1', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b279bc9b-94db-48be-b3ea-5e1219200f88', node_type=None, metadata={'page_label': '1', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, hash='6cb3f6149429b2a9c15988e08db61900f23d65c38b0e91ab55a6ebabb4c830e3')}, hash='47e8ba04489eeaf560e5298469cc01c94894ea5fecd5b58ca6c83990586bab1a', text='from Essays: First Series (1841)Type to enter textSelf-Reliance\n"Ne te quaesiveris extra." "Man is his own star; and the soul that can Render an honest and a perfect man, Commands all light, all inﬂuence, all fate; Nothing to him falls early or too late. Our acts our angels are, or good or ill, Our fatal shadows that walk by us still."       Epilogue to Beaumont and Fletcher\'s Honest Man\'s Fortune Cast the 

In [ ]:
%pprint

Pretty printing has been turned OFF


In [18]:
from llama_index.composability.joint_qa_summary import QASummaryQueryEngineBuilder

query_engine_builder = QASummaryQueryEngineBuilder()
query_engine = query_engine_builder.build_from_documents(documents)



INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 14199 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [20]:
summaries = []

for node in nodes:
    node = f"{node}" + "What is the summary of each paragraph here? (if there are 10 paragraphs, give me 10 summaries. One for each paragraph.)"
    response = query_engine.query(node)
    print(response, f"\n The text used was \n\n {node}")

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: This choice is most relevant to the question as it requires retrieval of specific context from documents..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 479 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1297 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



Paragraph 1: Self-Reliance is a concept that encourages individuals to trust their own judgement and rely on their own abilities.

Paragraph 2: Great men have always trusted their own judgement and accepted their place in the world, and we should do the same.

Paragraph 3: Nature provides examples of self-reliance in children, babes, and even animals, and we should not underestimate the power of youth. 
 The text used was 

 id_='2bdcb20e-9451-4423-b840-6a6993ada453' embedding=None metadata={'page_label': '1', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'} excluded_embed_metadata_keys=[] excluded_llm_metadata_keys=[] relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='59b57fa8-a94e-48bd-a0c8-c5466ca716bf', node_type=None, metadata={'page_label': '1', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, hash='6cb3f6149429b2a9c15988e08db61900f23d65c38b0e91ab55a6ebabb4c830e3')} hash='47e8ba04489eeaf560e5298469cc01c94894ea5fecd5b58ca6c83990586bab1a' text='

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Use this index for queries that require retrieval of specific context from documents..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 889 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2103 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



1. We should trust our own thoughts and ideas, and not be ashamed of them, even if they are different from the majority.

2. We should accept our place in life and use our unique talents to make a difference in the world.

3. Even children and animals can teach us to trust our instincts and not be swayed by the opinions of others. 
 The text used was 

 id_='62caf2f5-7056-4da9-85e2-f0f060f2c11e' embedding=None metadata={'page_label': '2', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'} excluded_embed_metadata_keys=[] excluded_llm_metadata_keys=[] relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='55ec9cc4-8b0d-4955-b590-5086229e258f', node_type=None, metadata={'page_label': '2', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, hash='88d9aa11df39227e95644e11fec0794414dc22896d946e7ffb540d0380d04dcc')} hash='7a725e9cf39faea6bc58ea2664ed629d8d7d68547ab2e6770754f84a30e4fa90' text="Self-Reliance - Ralph Waldo EmersonI read the other day some verses writt

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: This choice is most relevant to the question as it requires retrieval of specific context from documents..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 834 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2054 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


...

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Use this index for queries that require retrieval of specific context from documents..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 851 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2023 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



1. A boy is independent and gives an honest opinion, while a man is held accountable for his actions and words.
2. Society pressures people to conform, but self-reliance is the key to true success.
3. One should trust themselves and accept the place they are in, as great people have done before them. 
 The text used was 

 id_='4da828e4-dad3-422d-85bc-ba14dfaa0635' embedding=None metadata={'page_label': '4', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'} excluded_embed_metadata_keys=[] excluded_llm_metadata_keys=[] relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7007a79b-9c88-41f4-9a4d-d0c451a9812e', node_type=None, metadata={'page_label': '4', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, hash='bfbed38512b86497f11615faf12746f633159b7d6863858390e09c73430d132e')} hash='03029e26010b829f07b2f4586ec48e1922db4311d933e3de29308fc152518f9e' text='Self-Reliance - Ralph Waldo EmersonThe nonchalance of boys who are sure of a dinner, and would disdain a

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Use this index for queries that require retrieval of specific context from documents..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 936 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2190 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



1. No law should be followed except one's own nature, and one should not be swayed by external influences.
2. One should not be afraid to speak the truth, even if it is not popular.
3. Even children have a wisdom that adults should learn from, and one should trust themselves and accept their place in life. 
 The text used was 

 id_='28a1abe1-3783-45bf-8281-cb677d643448' embedding=None metadata={'page_label': '5', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'} excluded_embed_metadata_keys=[] excluded_llm_metadata_keys=[] relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cf1c72e4-6f2d-47be-ab12-e1eb117b0a5c', node_type=None, metadata={'page_label': '5', 'file_name': 'Self-Reliance - Ralph Waldo Emerson.pdf'}, hash='3ca176bdf6523d614d646cc57dd3ffc02dccd6615c35e7242045a4998563afaf')} hash='cd22c98d8994fa812a5f656786e93961f42ff7624fddf208ece14d45b8d1428d' text='Self-Reliance - Ralph Waldo Emersontraditions, if I live wholly from within? my friend suggested, 

KeyError: 'choice'

In [3]:
# create evaluation of accuracy